In [42]:
# path 설정
import sys
sys.path.append('/Users/jtchoi/Library/CloudStorage/GoogleDrive-jungtaek0227@gmail.com/My Drive/quant/Quant-Project/quant/')

# 제작한 모듈들 임포트
from price.price_processing import *
from strategy.factors.Momentum import MomentumFactor
from strategy.optimize.CrossSectional import *
from strategy.optimize.TimeSeries import *

# S&P500에 속한 기업들 티커를 받기 위한 모듈 임포트
import bs4 as bs
import requests
import yfinance as yf
import datetime

# 기타 모듈 임포트
import matplotlib.pyplot as plt
from collections import Counter
import datetime as dt
import pickle5 as pickle
import seaborn as sns
sns.set()

In [43]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

tickers_list = [s.replace('\n', '') for s in tickers]

In [44]:
start = datetime.datetime(2000,1,1)
end = datetime.datetime(2021,12,31)
df = yf.download(tickers_list, start=start, end=end)
sp_price = df['Adj Close']

sp_price.index = pd.to_datetime(pd.to_datetime(sp_price.index).date)
sp_price.index.name = 'date_time'

sp_price

[*********************100%***********************]  503 of 503 completed

4 Failed downloads:
- BRK.B: No timezone found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
- CEG: Data doesn't exist for startDate = 946702800, endDate = 1640926800
- GEHC: Data doesn't exist for startDate = 946702800, endDate = 1640926800


,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date_time,,,,,,,,,,,,,,,,,,,,,
2000-01-03,44.001263,NaN,NaN,0.851942,NaN,2.998109,9.221025,1.277778,NaN,16.274673,...,NaN,7.294351,19.130629,6.772070,NaN,4.792224,NaN,25.027779,37.955730,NaN
2000-01-04,40.640045,NaN,NaN,0.780115,NaN,2.793419,8.957567,1.270833,NaN,14.909400,...,NaN,7.462309,18.764206,6.772070,NaN,4.695899,NaN,24.666668,36.117775,NaN
2000-01-05,38.119148,NaN,NaN,0.791530,NaN,3.010151,8.941103,1.388889,NaN,15.204173,...,NaN,7.750247,19.787151,6.889141,NaN,4.719981,NaN,25.138889,36.075035,NaN
2000-01-06,36.667717,NaN,NaN,0.723033,NaN,3.238922,9.253956,1.375000,NaN,15.328289,...,NaN,7.678262,20.810095,6.898144,NaN,4.679844,NaN,23.777779,36.587959,NaN
2000-01-07,39.723362,NaN,NaN,0.757282,NaN,3.660342,9.352753,1.451389,NaN,16.072987,...,NaN,7.678262,20.749043,6.880133,NaN,4.575491,NaN,23.513889,36.673447,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-23,156.562347,18.260000,224.063004,175.262802,126.892914,127.924423,136.246506,43.480000,396.261017,569.619995,...,88.879997,64.804298,58.691692,54.536747,115.991066,132.787628,122.040504,582.409973,60.688782,240.179321
2021-12-27,157.494995,18.170000,228.281128,179.289459,128.151443,130.265030,138.498352,43.930000,408.070953,577.679993,...,87.580002,65.008606,59.528500,55.048370,116.770920,135.407288,123.061127,606.330017,61.321362,244.140915
2021-12-28,157.931503,18.540001,229.854492,178.255417,128.132385,130.719345,137.529083,44.270000,408.012024,569.359985,...,86.459999,65.786919,59.336132,55.127079,117.985115,135.377853,123.446266,597.320007,61.418682,241.902618


In [45]:
sp_price = sp_price.dropna(axis=1)
sp_price.columns.values

array(['A', 'AAPL', 'ABC', 'ABT', 'ACGL', 'ADBE', 'ADI', 'ADM', 'ADP',
       'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AJG', 'AKAM', 'ALB',
       'ALK', 'ALL', 'AMAT', 'AMD', 'AME', 'AMGN', 'AMT', 'AMZN', 'ANSS',
       'AON', 'AOS', 'APA', 'APD', 'APH', 'ARE', 'ATO', 'ATVI', 'AVB',
       'AVY', 'AXP', 'AZO', 'BA', 'BAC', 'BALL', 'BAX', 'BBWI', 'BBY',
       'BDX', 'BEN', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLK', 'BMY',
       'BRO', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CAT', 'CB', 'CCI',
       'CCL', 'CDNS', 'CHD', 'CHRW', 'CI', 'CINF', 'CL', 'CLX', 'CMA',
       'CMCSA', 'CMI', 'CMS', 'CNP', 'COF', 'COO', 'COP', 'COST', 'CPB',
       'CPRT', 'CPT', 'CSCO', 'CSGP', 'CSX', 'CTAS', 'CTRA', 'CTSH',
       'CVS', 'CVX', 'D', 'DD', 'DE', 'DGX', 'DHI', 'DHR', 'DIS', 'DISH',
       'DLTR', 'DOV', 'DRI', 'DTE', 'DUK', 'DVA', 'DVN', 'DXC', 'EA',
       'EBAY', 'ECL', 'ED', 'EFX', 'EIX', 'EL', 'EMN', 'EMR', 'EOG',
       'EQR', 'EQT', 'ES', 'ESS', 'ETN', 'ETR', 'EVRG', 'EXC', 'EXPD',

In [46]:
# 가격 데이터 로드
with open('../../part07.-퀀트-트레이딩-실무-실습-파일/chapter3/피클파일/spx_price.pkl', 'rb') as f:
    spx_index = pickle.load(f)
    spx_index.index = pd.to_datetime(spx_index.index)
    
with open('../../part07.-퀀트-트레이딩-실무-실습-파일/chapter3/피클파일/commodity_price.pkl', 'rb') as f:
    commodity_price = pickle.load(f)
    commodity_price.index = pd.to_datetime(commodity_price.index)
    
with open('../../part07.-퀀트-트레이딩-실무-실습-파일/chapter3/피클파일/usdkrw_price.pkl', 'rb') as f:
    usdkrw_price = pickle.load(f)
    usdkrw_price.index = pd.to_datetime(usdkrw_price.index)
    
with open('../../part07.-퀀트-트레이딩-실무-실습-파일/chapter3/피클파일/oecd_cli.pkl', 'rb') as f:
    oecd_cli_df = pickle.load(f)
    oecd_cli_df.index = pd.to_datetime(oecd_cli_df.index)
    # 선견편향 제거
    oecd_cli_df = oecd_cli_df.shift(1)
    # 시점별 국가 수 계산
    noc = len(oecd_cli_df.columns) - oecd_cli_df.diff().isna().sum(axis=1)
    # 확산 지수
    diffusion_index = (oecd_cli_df.diff() > 0).sum(axis=1) / noc

bond_tickers = ['TLT', 'IEF']
bond_price = yf.Tickers(bond_tickers)
bond_price = bond_price.history(start='2010-01-01', actions=False)
bond_price.drop(['Open', 'High', 'Low', 'Volume'], inplace=True, axis=1)
bond_price = bond_price.droplevel(0, axis=1)#.resample('W').last()
bond_price.index = pd.to_datetime(bond_price.index.date)

[*********************100%***********************]  2 of 2 completed


In [47]:
price_universe_df = pd.concat([spx_index, sp_price, commodity_price, usdkrw_price, bond_price], axis=1)
price_universe_df = price_universe_df.dropna(axis=0)

In [48]:
price_universe_df
price_universe_df.to_csv('price_universe.csv')

### 거시경제 지표

In [8]:
with open('../../part07.-퀀트-트레이딩-실무-실습-파일/chapter3/피클파일/oecd_cli.pkl', 'rb') as f:
    oecd_cli_df = pickle.load(f)
    oecd_cli_df.index = pd.to_datetime(oecd_cli_df.index)
    # 선견편향 제거
    oecd_cli_df = oecd_cli_df.shift(1)
    # 시점별 국가 수 계산
    noc = len(oecd_cli_df.columns) - oecd_cli_df.diff().isna().sum(axis=1)
    # 확산 지수
    diffusion_index = (oecd_cli_df.diff() > 0).sum(axis=1) / noc

In [9]:
with open('../../part07.-퀀트-트레이딩-실무-실습-파일/chapter3/피클파일/factor.pkl', 'rb') as f:
    df = pickle.load(f)
    regime = df.iloc[:, :5]
    regime.index = pd.to_datetime(regime.index)

In [20]:
regime.columns.values

array(['Growth', 'Inflation', 'Volatilities', 'Funding Liquidity',
       'Market Liquidity'], dtype=object)

In [11]:
diffusion_index = pd.DataFrame({'Diffusion': diffusion_index})
diffusion_index

,Diffusion
1955-01-31,NaN
1955-02-28,NaN
1955-03-31,1.000000
1955-04-30,1.000000
1955-05-31,1.000000
...,...
2022-05-31,0.088235
2022-06-30,0.029412
2022-07-31,0.000000
2022-08-31,0.000000


In [12]:
econ_regime = pd.concat([diffusion_index, regime], axis=1)
#econ_regime = econ_regime.rename(columns={0: 'Diffusion'})
econ_regime.index = pd.to_datetime(econ_regime.index)
econ_regime.index = pd.to_datetime(econ_regime.index.date)
econ_regime.dropna(axis=0, inplace=True)

econ_regime.to_csv('econ_regime.csv')

In [24]:
col_info_dict = {'equity': sp_price,
                'sp500 index': spx_index,
                'commodity': commodity_price,
                'fx': usdkrw_price,
                'bond etf': bond_price,
                'regime': econ_regime,
                }

In [25]:
def make_col_infos(df, name): 
    df = pd.DataFrame(df.columns.values)
    df = df.rename(index = lambda x: name)
    return df

In [36]:
df_list = []

for key, df in col_info_dict.items():
    df_list.append(make_col_infos(df, name=key))


In [39]:
price_info_df = pd.concat(df_list)
price_info_df = price_info_df.rename(columns={0:'price info'})


In [41]:
price_info_df.to_csv('price_info_df.csv')

In [ ]:
target_rebal = rebal_dates(price_universe_df, 'month')
target_rebal

In [ ]:
rebal_price_df = price_on_rebal(price_universe, target_rebal)
rebal_price_df

In [ ]:
test_mt_factor = MomentumFactor(rebal_price=rebal_price_df, lookback_window=1, n_sel=3)
test_rel_signal = test_mt_factor.relative_momentum()
display(test_rel_signal)

In [ ]:
cs_weight = Equalizer(signal=test_rel_signal, rebal_price=rebal_price_df, param=12)
beta_weight = cs_weight.beta()
ew_weight = cs_weight.ew()

In [ ]:
ew_weight = ew_weight.fillna(0)

In [ ]:
# 횡적 자산배분의 결과 확인
port_val = calculate_portvals(price_df=price_universe, weight_df=beta_weight)
port_val

port_rets = get_daily_rets(port_val)
port_rets

port_cum_rets = get_cum_rets(port_rets)
port_cum_rets.plot()

In [ ]:
# TimeSeries의 변수로 들어갈 포트폴리오 리턴을 만든다. 
def port_val(weight_df, rebal_price_df): 
    
    # 포트폴리오 수익률 데이터프레임
    rets = rebal_price_df.pct_change().fillna(0)
    port_rets = weight_df * rets
    port_rets = port_rets.sum(axis=1)
    port_cum_rets = (1 + port_rets).cumprod() - 1
    
    return port_rets

port_rets = port_val(beta_weight, rebal_price_df)
type(port_rets)

In [ ]:
port_rets

In [ ]:
target = 0.5
ts_weight = pd.DataFrame({'PORTFOLIO': target, 'CASH': 1 - target}, index=port_rets.index)
ts_weight

In [ ]:
# 종적 자산배분 
ew_weights, ew_ew_weight = TimeSeries(port_rets=port_rets, cs_weight=beta_weight, call_method='ew', param=12).run()
vt_weights, ew_vt_weight = TimeSeries(port_rets=port_rets, cs_weight=beta_weight, call_method='vt', param=12).run()
cvt_weights, ew_cvt_weight = TimeSeries(port_rets=port_rets, cs_weight=beta_weight, call_method='cvt', param=12).run()
kl_weights, kl_cvt_weight = TimeSeries(port_rets=port_rets, cs_weight=beta_weight, call_method='kl', param=12).run()
cppi_weights, cppi_cvt_weight = TimeSeries(port_rets=port_rets, cs_weight=beta_weight, call_method='cppi', param=12).run()
# vt_weights = test_ts.vt()
# cvt_weights = test_ts.cvt()
# kl_weights = test_ts.kl()
# cppi_weights = test_ts.cppi()

In [ ]:
ew_weights

In [ ]:
cvt_weights

In [ ]:
# 시계열 플랏 시각화
plt.figure(figsize=(20, 5))
plt.stackplot(cvt_weights.index, cvt_weights.T, labels=cvt_weights.columns, colors=['steelblue', 'silver'])
plt.legend(loc='upper left')
plt.title('CVT Weights')
plt.xlabel('Date')
plt.ylabel('Weights')

In [ ]:
ts_models = {'vt':vt_weights, 
            'cvt':cvt_weights, 
            'kl': kl_weights, 
            'cppi': cppi_weights
            }

In [ ]:
# 시계열 플랏 시각화
def plot_ts_weight(weight_df, name):
    plt.figure(figsize=(20, 5))
    plt.stackplot(weight_df.index, weight_df.T, labels=weight_df.columns, colors=['steelblue', 'silver'])
    plt.legend(loc='upper left')
    plt.title(f'{name} Weights')
    plt.xlabel('Date')
    plt.ylabel('Weights')
    
for name, ts in ts_models.items():
    plot_ts_weight(ts, name)

In [ ]:
price_universe = add_cash(price_universe, 252, 0.04)

In [ ]:
# 종적 자산배분의 결과 확인
port_val = calculate_portvals(price_df=price_universe, weight_df=ew_cvt_weight)
port_val

port_rets = get_daily_rets(port_val)
port_rets

port_cum_rets = get_cum_rets(port_rets)
port_cum_rets.plot()